## Next.js API

To create your own API endpoint in Next.js, you have to add folder api in app folder. For example you want to have endpoint

.../api/blog

Create blog folder in the api folder. Create file route.* in the folder. This is how we tell Next.js to treat it as an API endpoint.

In [ ]:
## route.tsx (in @/app/api/blog)
import { Post } from "@/lib/models";
import { connectToDb } from "@/lib/utils"  # connectToDb() - please check MongoDB notes how to connect with DB (chapter 01).
import { NextResponse } from "next/server";

export const GET = async (request) => {
    try {
        connectToDb();
        const posts = await Post.find();
        return NextResponse.json(posts);
    } catch (err) {
        throw new Error("Failed to fetch posts!")
    }
}

Now we can fetch our data. In blog page file we can go back to fetching with an API.

In [ ]:
import { PostCard } from '@/components/postCard/PostCard';
import styles from './blog.module.css';
import {getPosts} from '@/lib/data'

const getData = async () => {
  const res = await fetch("http://localhost:3000/api/blog")  # we fetch now from our local api endpoint

  if (!res.ok) {
    throw new Error("Something went wrong")
  }

  return res.json()
}

const BlogPage = async () => {

  const posts = await getData();

  return (
    <div className={styles.container}>
      {posts && posts.map((post: {userId: number, slug: number, title: string, body: string, createdAt: Date}) => {
        return (
          <div className={styles.post} key={post.slug}>
            <PostCard post={post}/>
          </div>
        )
      })}

    </div>
  )
}

export default BlogPage

We can make a dynamic endpoints in the same way as for routes. We have to put the folder name in the brackets.
So .../api/blog/[slug]. So, we can fetch a single post data.

In [ ]:
import { Post } from "@/lib/models";
import { connectToDb } from "@/lib/utils";
import { NextResponse } from "next/server";

export const GET =  async (request: any, {params}: {params: any}) => { ## added params, after request!
    const {slug} = params;

    try {
        connectToDb();
        const post = await Post.findOne({slug});  # used .findOne to find exactly the one we are looking for
        return NextResponse.json(post);
    } catch (err) {
        console.log(err)
        throw new Error("Failed to fetch post!")
    }
}

Now we can fetch data from an API and use the data in our page.

In [ ]:
import Image from "next/image"
import styles from "./singlePost.module.css"
import PostUser from "@/components/postUser/PostUser"
import { Suspense } from "react"
import { getPost } from "@/lib/data";

const getSinglePost = async (slug: string) => {
  const res = await fetch(`http://localhost:3000/api/blog/${slug}`) # we fetch now from our local api endpoint

  if (!res.ok) {
    throw new Error("Something went wrong")
  }

  return res.json();
}


export const generateMetadata = async ({params} : {params: {slug: string}}) => {
  const {slug} = params;

  const post = await getPost(slug)

  return {
    title: post.title,
    description: post.desc,
  }
}

const SinglePostPage = async ({params} : {params: {slug: string}}) => {

  const {slug} = params;

  const post = await getSinglePost(slug);


  return (
    <div className={styles.container}>
        ...
    </div>
  )
}

export default SinglePostPage

We can already add a delete method to the post, but we are not going to use it on our pages yet.

In [ ]:
## route.tsx (in api/blog/[slug])

import { Post } from "@/lib/models";
import { connectToDb } from "@/lib/utils";
import { NextResponse } from "next/server";

export const DELETE =  async (request: any, {params}: {params: any}) => { ## added params, after request!
    const {slug} = params;

    try {
        connectToDb();
        wait Post.deleteOne({slug});  # used .deleteOne to delete exactly the one we are looking for
        return NextResponse.json("Post deleted.");
    } catch (err) {
        console.log(err)
        throw new Error("Failed to fetch post!")
    }
}